In [3]:
import gymnasium as gym
import numpy as np
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy, BaseFeaturesExtractor
from stable_baselines3.common.callbacks import EvalCallback
import pandas as pd
from tensorflow import keras

# ------------------ Custom Environment Wrapper ------------------

class CustomHighwayEnv(gym.Wrapper):
    def step(self, action):
        obs, reward, done, info,_ = self.env.step(action)
        if done:
            reward = +1000
        else:
            reward = -50
        return obs, reward, done, info,_  

# ------------------ Behavior Cloning ------------------

class BehaviorCloningPolicy:
    def __init__(self, model_path):
        self.model = keras.models.load_model(model_path)

    def predict(self, ego_vehicle, vehicles):
        obs = [self.vehicle_to_observation(ego_vehicle)]
        for vehicle in vehicles:
            obs.append(self.vehicle_to_observation(vehicle))
        while len(obs) < 5:
            obs.append([0, 0, 0, 0, 0])  # Padding
        obs = np.array(obs).reshape(1, 5, 5)
        action_probs = self.model.predict(obs)
        return np.argmax(action_probs[0])

    @staticmethod
    def vehicle_to_observation(vehicle):
        return [
            1.0,
            vehicle.position[0] / 100.0,
            vehicle.position[1] / 5.0,
            vehicle.speed / 30.0,
            vehicle.heading / (2 * np.pi)
        ]

# ------------------ Custom Network and Policy ------------------

class D2RLNetwork(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim: int = 256):
        super(D2RLNetwork, self).__init__(observation_space, features_dim)
        self.flatten = nn.Flatten()
        self.hidden = nn.Sequential(
            nn.Linear(np.prod(observation_space.shape), features_dim),
            nn.ReLU()
        )
        self.d2rl1 = nn.Sequential(
            nn.Linear(features_dim, features_dim),
            nn.ReLU()
        )
        self.d2rl2 = nn.Sequential(
            nn.Linear(features_dim, features_dim),
            nn.ReLU()
        )

    def forward(self, observations):
        x = self.flatten(observations)
        x1 = self.hidden(x)
        x2 = self.d2rl1(x1)
        x3 = self.d2rl2(x1 + x2)
        return x3

class D2RLPolicy(ActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super(D2RLPolicy, self).__init__(*args, **kwargs, features_extractor_class=D2RLNetwork, features_extractor_kwargs=dict(features_dim=256))


In [4]:

# ------------------ Main Execution ------------------

if __name__ == "__main__":
    # Create environment
    base_env = gym.make("highway-fast-v0")
    env = CustomHighwayEnv(gym.make("highway-fast-v0", render_mode="rgb_array"))

    # Load the pretrained behavior cloning model
    model_path = "C:\\Users\\Ram\\highway\\perturbed_models\\perturbed_model_1.keras"
    bc_policy = BehaviorCloningPolicy(model_path)

    # Callbacks for evaluation during training and tensorboard
    eval_callback = EvalCallback(env, best_model_save_path='./logs/best_model',
                                 log_path='./logs/results', eval_freq=100, n_eval_episodes=5)

    # Train PPO with the custom D2RL policy
    model = PPO(D2RLPolicy, env, verbose=1, tensorboard_log="./tensorboard/")
    model.learn(total_timesteps=1000, callback=eval_callback)

    # Test the trained model and save rewards to CSV
    model = PPO.load("C:\\Users\\Ram\\highway\\logs\\best_model\\best_model.zip", env=env)

    episode_data = {}
    for ep in range(10000):
        obs = env.reset()
        if len(obs) > 1:
            obs = obs[0]
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, _,_ = env.step(action)
            episode_reward += reward
            env.render()
        episode_data[ep+1] = episode_reward

    # Save episode rewards to CSV
    pd.DataFrame({"Episode ": list(episode_data.keys()), "Reward": list(episode_data.values())}).to_csv("d2rl_PPO.csv", index=False)
    print("Training and testing complete!")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard/PPO_1


c:\Users\Ram\anaconda3.0\envs\highwayenv\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
c:\Users\Ram\anaconda3.0\envs\highwayenv\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
c:\Users\Ram\anaconda3.0\envs\highwayenv\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, 

Eval num_timesteps=100, episode_reward=630.00 +/- 150.33
Episode length: 8.40 +/- 3.01
---------------------------------
| eval/              |          |
|    mean_ep_length  | 8.4      |
|    mean_reward     | 630      |
| time/              |          |
|    total_timesteps | 100      |
---------------------------------
New best mean reward!


c:\Users\Ram\anaconda3.0\envs\highwayenv\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=200, episode_reward=800.00 +/- 154.92
Episode length: 5.00 +/- 3.10
---------------------------------
| eval/              |          |
|    mean_ep_length  | 5        |
|    mean_reward     | 800      |
| time/              |          |
|    total_timesteps | 200      |
---------------------------------
New best mean reward!


...

c:\Users\Ram\anaconda3.0\envs\highwayenv\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=1900, episode_reward=400.00 +/- 320.94
Episode length: 13.00 +/- 6.42
---------------------------------
| eval/              |          |
|    mean_ep_length  | 13       |
|    mean_reward     | 400      |
| time/              |          |
|    total_timesteps | 1900     |
---------------------------------


c:\Users\Ram\anaconda3.0\envs\highwayenv\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=2000, episode_reward=640.00 +/- 159.37
Episode length: 8.20 +/- 3.19
---------------------------------
| eval/              |          |
|    mean_ep_length  | 8.2      |
|    mean_reward     | 640      |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 11.2     |
|    ep_rew_mean     | 456      |
| time/              |          |
|    fps             | 41       |
|    iterations      | 1        |
|    time_elapsed    | 49       |
|    total_timesteps | 2048     |
---------------------------------
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Training and testing complete!


: 